In [ ]:
county_year = pd.read_csv("county_unemployment_1990_2022.csv", skiprows=[1], skipfooter=3, engine='python')
county_year = county_year.sort_values('year')
county_year = county_year.dropna(subset=['year'])
county_year = county_year.drop(['laus_code', 'state_fips', 'county_fips', 'labor_force', 'employed', 'unemployed'], axis=1)
county_year

In [ ]:
# Remove ".0" from the end of fips strings
county_year['fips'] = county_year['fips'].str.rstrip('.0')
county_year

In [ ]:
# Split fips into two parts
fips_parts = county_year['fips'].str.split('.')

# Add leading zeros to the first part of fips where necessary
fips_parts_1 = fips_parts.str[0].str.pad(width=2, fillchar='0')

# Add leading zeros to the second part of fips where necessary and remove leading zeros if len > 3
fips_parts_2 = fips_parts.str[1]
fips_parts_2[fips_parts_2.str.len() > 3] = fips_parts_2.str.lstrip('0')
fips_parts_2[fips_parts_2.str.len() < 3] = fips_parts_2.str.pad(width=3, fillchar='0')

# Concatenate the two parts of fips
county_year['fips'] = fips_parts_1 + fips_parts_2
county_year

In [ ]:
import numpy as np
county_year['unemployment_rate'] = county_year['unemployment_rate'].replace('N.A.', np.nan)
county_year['unemployment_rate'] = county_year['unemployment_rate'].astype(float)
county_year['year'] = county_year['year'].astype(str)

In [2]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from urllib.request import urlopen
import json

In [4]:
# Load state_year.csv data
county_year = pd.read_csv("county_year_unemployment.csv")
county_year.dtypes

county                object
year                 float64
unemployment_rate     object
state                 object
fips                   int64
dtype: object

In [5]:
county_year['fips'] = county_year['fips'].astype(str)
county_year['fips'] = county_year['fips'].apply(lambda x: '0' + x if len(x) == 4 else x)
county_year['fips'].str.len().unique()

array([5])

In [6]:
county_year

,county,year,unemployment_rate,state,fips
0,Gallatin County,1990.0,4.8,MT,30031
1,Cleveland County,1990.0,4.3,OK,40027
2,Coal County,1990.0,12.3,OK,40029
3,Comanche County,1990.0,6.9,OK,40031
4,Cotton County,1990.0,11.3,OK,40033
...,...,...,...,...,...
106181,Montgomery County,2022.0,4.5,KY,21173
106182,Morgan County,2022.0,5.3,KY,21175
106183,Muhlenberg County,2022.0,5.7,KY,21177
106184,Marshall County,2022.0,4.1,KY,21157


In [7]:
#Due to 1976 - 2022 datasets are too large to process, I select only 2 years to gain a result.
county_year_2021_2022 = county_year[(county_year['year'] >= 2021) & (county_year['year'] <= 2022)]
county_year_2021_2022

,county,year,unemployment_rate,state,fips
99746,Meade County,2021.0,5.0,KY,21163
99747,McLean County,2021.0,4.1,KY,21149
99748,Mason County,2021.0,5.2,KY,21161
99749,Martin County,2021.0,8.6,KY,21159
99750,Marshall County,2021.0,4.4,KY,21157
...,...,...,...,...,...
106181,Montgomery County,2022.0,4.5,KY,21173
106182,Morgan County,2022.0,5.3,KY,21175
106183,Muhlenberg County,2022.0,5.7,KY,21177
106184,Marshall County,2022.0,4.1,KY,21157


In [ ]:
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

# Create the map with a slider
fig = px.choropleth(
    county_year_2021_2022,
    geojson=counties,
    locations='fips',
    color='unemployment_rate',
    color_continuous_scale='reds',
    scope='usa',
    animation_frame='year',
    hover_name='county',
    hover_data=['unemployment_rate'],
    labels={'unemployment_rate': 'Unemployment Rate'},
    title='US County Unemployment Rates (2018-2022)',
)

# Update the layout
fig.update_layout(
    title=dict(x=0.5, xanchor="center"),
    margin={"r":0,"t":40,"l":0,"b":0},
    coloraxis_colorbar=dict(title="Unemployment Rate"),
)

fig.show()

In [ ]:
# Create the map with a slider
"""fig_counties = px.choropleth_mapbox(
    county_year,
    geojson=counties,
    locations='fips',
    color='unemployment_rate',
    color_continuous_scale='Viridis',
    animation_frame='year',
    hover_name='county',
    hover_data=['unemployment_rate'],
    labels={'unemployment_rate': 'Unemployment Rate'},
    title='US County Unemployment Rates (2017-2022)',
    range_color=(0, 12),
    mapbox_style="carto-positron",
    zoom=3, center={"lat": 37.0902, "lon": -95.7129},
    opacity=0.5,
)

# Update the layout
fig_counties.update_layout(
    title=dict(x=0.5, xanchor="center"),
    margin={"r":0,"t":40,"l":0,"b":0},
    coloraxis_colorbar=dict(title="Unemployment Rate"),
)

fig_counties.show()"""